<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>Audience Score 
</b></div>

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Read data</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import json
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from scipy.sparse import hstack
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input, Embedding, GRU, Dense, Dropout, 
    Concatenate, Bidirectional, BatchNormalization,
    GlobalAveragePooling1D, GlobalMaxPooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import joblib


In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [4]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [7]:
print("Movies:", movies.columns.tolist())
print("Credits:", credits.columns.tolist())

Movies: ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']
Credits: ['movie_id', 'title', 'cast', 'crew']


In [8]:
movies.shape

(4803, 20)

In [9]:
credits.shape

(4803, 4)

In [10]:
df = movies.merge(credits, on='title', how='inner')

In [11]:
df.shape

(4809, 23)

In [12]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Handle missing values and duplicates</div>

In [13]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

In [14]:
df = df.drop(columns=['homepage', 'tagline'])

In [15]:
df.dropna(inplace=True)

In [16]:
df.isna().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
title                   0
vote_average            0
vote_count              0
movie_id                0
cast                    0
crew                    0
dtype: int64

In [17]:
df.duplicated().sum()

np.int64(0)

In [18]:
df['vote_average'].describe()

count    4805.000000
mean        6.094527
std         1.187720
min         0.000000
25%         5.600000
50%         6.200000
75%         6.800000
max        10.000000
Name: vote_average, dtype: float64

In [19]:
# plt.figure(figsize=(10, 5))
# plt.subplot(1, 2, 1)
# plt.hist(df['vote_average'], bins=30, edgecolor='black')
# plt.xlabel('Vote Average (Audience Score)')
# plt.ylabel('Count')
# plt.title('Audience Score distribution')

# plt.subplot(1, 2, 2)
# plt.boxplot(df['vote_average'])
# plt.ylabel('Vote Average')
# plt.title('Boxplot for Score')
# plt.tight_layout()
# plt.show()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Audience Score Distribution", "Boxplot for Score")
)

hist_fig = px.histogram(
    df,
    x="vote_average",
    nbins=30
)

for trace in hist_fig.data:
    fig.add_trace(trace, row=1, col=1)

box_fig = px.box(
    df,
    y="vote_average"
)

for trace in box_fig.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_xaxes(title_text="Vote Average (Audience Score)", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.update_yaxes(title_text="Vote Average", row=1, col=2)

fig.update_layout(
    width=1000,
    height=500,
    showlegend=False
)

fig.show()


In [20]:
df_clean = df.copy()

In [21]:
df_clean.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count',
       'movie_id', 'cast', 'crew'],
      dtype='object')

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Feature Engineering & Selection</div>

In [22]:
def extract_genres(genres_str):
    try:
        genres_list = literal_eval(genres_str) # It converts a String that represents a Python structure.
        return [g['name'] for g in genres_list]
    except:
        return []

df_clean['genres_list'] = df_clean['genres'].apply(extract_genres) # It contains a ratings list for each film.
df_clean['num_genres'] = df_clean['genres_list'].apply(len) # It contains the number of genres for each film.

In [23]:
def extract_keywords(keywords_str):
    try:
        keywords_list = literal_eval(keywords_str)
        return [k['name'] for k in keywords_list]
    except:
        return []

df_clean['keywords_list'] = df_clean['keywords'].apply(extract_keywords) # It contains a keywords list for each film.
df_clean['num_keywords'] = df_clean['keywords_list'].apply(len) # It contains the number of keywords for each film.

In [24]:
def count_cast(cast_str):
    try:
        cast_list = literal_eval(cast_str)
        return len(cast_list)
    except:
        return 0

df_clean['num_cast'] = df_clean['cast'].apply(count_cast) # It contains the number of cast members for each film.

In [25]:
df_clean['release_date'] = pd.to_datetime(df_clean['release_date'], errors='coerce')
df_clean['release_year'] = df_clean['release_date'].dt.year
df_clean['release_month'] = df_clean['release_date'].dt.month

In [26]:
df_clean['overview'] = df_clean['overview'].fillna('') # Fill missing overviews with empty strings
df_clean['overview_length'] = df_clean['overview'].apply(len) # It contains the length of the overview text.

In [27]:
df_clean.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count',
       'movie_id', 'cast', 'crew', 'genres_list', 'num_genres',
       'keywords_list', 'num_keywords', 'num_cast', 'release_year',
       'release_month', 'overview_length'],
      dtype='object')

In [28]:
important_cols = [
    'title', 'vote_average', 'overview', 'budget', 'revenue', 
    'runtime', 'popularity', 'vote_count', 'release_year', 
    'release_month', 'num_genres', 'num_keywords', 'num_cast',
    'genres_list', 'keywords_list', 'overview_length'
]

df_final = df_clean[important_cols].copy()

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">EDA</div>

In [31]:
fig1 = px.histogram(
    df_final, 
    x='vote_average', 
    nbins=40,
    title='Audience Score distribution(vote_average)',
    labels={'vote_average': 'Audience Score'},
    color_discrete_sequence=['#1f77b4']
)
fig1.update_layout(
    bargap=0.1,
    xaxis_title='Audience Score',
    yaxis_title='عدد الأفلام',
    template='plotly_white'
)
fig1.show()

In [32]:
fig2 = px.scatter(
    df_final.dropna(subset=['release_year']),
    x='release_year',
    y='vote_average',
    color='vote_average',
    color_continuous_scale='Viridis',
    title='Audience Score by Release Year',
    labels={'release_year': 'Release Year', 'vote_average': 'Audience Score'},
    hover_name='title',
    trendline='ols'
)
fig2.update_layout(template='plotly_white')
fig2.show()

In [33]:
all_genres = [genre for sublist in df_final['genres_list'] for genre in sublist]
genre_counts = pd.Series(all_genres).value_counts().head(10)

fig3 = px.bar(
    y=genre_counts.index,
    x=genre_counts.values,
    orientation='h',
    title='Top 10 Most Common Movie Genres',
    labels={'x': 'Number of Movies', 'y': 'Genre'},
    color=genre_counts.values,
    color_continuous_scale='Blues'
)
fig3.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    template='plotly_white'
)
fig3.show()

In [34]:
fig4 = px.scatter(
    df_final,
    x='budget',
    y='vote_average',
    color='popularity',
    size='vote_count',
    title='Budget vs Audience Score',
    labels={'budget': 'Budget (USD)', 'vote_average': 'Audience Score'},
    hover_name='title',
    log_x=True 
)
fig4.update_layout(template='plotly_white')
fig4.show()

In [35]:
monthly_avg = df_final.groupby('release_month')['vote_average'].mean().reset_index()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig5 = px.bar(
    monthly_avg,
    x='release_month',
    y='vote_average',
    title='Average Audience Score by Release Month',
    labels={'release_month': 'Month', 'vote_average': 'Average Score'},
    color='vote_average',
    color_continuous_scale='RdYlGn',
    text=monthly_avg['vote_average'].round(2)
)
fig5.update_xaxes(tickvals=list(range(1,13)), ticktext=month_names)
fig5.update_layout(template='plotly_white')
fig5.show()

In [36]:
corr_features = ['vote_average', 'budget', 'revenue', 'runtime', 
                 'popularity', 'vote_count', 'overview_length', 
                 'num_genres', 'num_keywords', 'num_cast']

corr_matrix = df_final[corr_features].corr()

fig6 = px.imshow(
    corr_matrix,
    text_auto='.2f',
    aspect="auto",
    title='correlation Heatmap between Numerical Features and Audience Score',
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1
)
fig6.update_xaxes(side="top")
fig6.update_layout(template='plotly_white')
fig6.show()

In [37]:
fig7 = px.scatter(
    df_final,
    x='revenue',
    y='vote_average',
    color='budget',
    size='popularity',
    title='Revenue vs Audience Score',
    labels={'revenue': 'Revenue', 'vote_average': 'Audience Score'},
    hover_name='title',
    log_x=True
)
fig7.update_layout(template='plotly_white')
fig7.show()

In [38]:
fig8 = px.scatter(
    df_final,
    x='popularity',
    y='vote_average',
    size='vote_count',
    color='vote_count',
    color_continuous_scale='Plasma',
    title='Popularity vs Audience Score (Size of point and color = number of votes)',
    labels={'popularity': 'Popularity', 'vote_average': 'Audience Score'},
    hover_name='title'
)
fig8.update_layout(template='plotly_white')
fig8.show()

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Text Cleaning</div>

In [234]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moshe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\moshe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\moshe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [235]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization
    tokens = text.split()

    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]

    return " ".join(tokens)


In [236]:
df_final['overview_cleaned'] = df_final['overview'].apply(clean_text)

In [237]:
df_final.head(2)

,title,vote_average,overview,budget,revenue,runtime,popularity,vote_count,release_year,release_month,num_genres,num_keywords,num_cast,genres_list,keywords_list,overview_length,overview_cleaned
0,Avatar,7.2,"In the 22nd century, a paraplegic Marine is di...",237000000,2787965087,162.0,150.437577,11800,2009,12,4,21,83,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",175,century paraplegic marine dispatched moon pand...
1,Pirates of the Caribbean: At World's End,6.9,"Captain Barbossa, long believed to be dead, ha...",300000000,961000000,169.0,139.082615,4500,2007,5,3,16,34,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",176,captain barbossa long believed dead come back ...


<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">TF-IDF (Term Frequency-Inverse Document Frequency)</div>

In [238]:
tfidf = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2),
    min_df=5
)
X = tfidf.fit_transform(df_final['overview_cleaned'])

In [239]:
num_cols = [
    'budget', 'revenue', 'runtime', 'popularity',
    'vote_count', 'release_year', 'release_month',
    'num_genres', 'num_keywords', 'num_cast', 'overview_length'
]

X_num = df_final[num_cols]

In [240]:
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Split data</div>

In [241]:
x = hstack([X, X_num_scaled])
y = df_final['vote_average']

In [242]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b> Training and prediction using ML models
</b></div>

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Training with Ridge</div>

In [243]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [244]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [245]:
print('MAE_train:', mean_absolute_error(y_train, y_train_pred))
print("MAE_test:", mean_absolute_error(y_test, y_test_pred))

MAE_train: 0.5100460053786072
MAE_test: 0.6893421460188089


In [246]:
print('MSE_train:', mean_squared_error(y_train, y_train_pred))
print("MSE_test:", mean_squared_error(y_test, y_test_pred))

MSE_train: 0.5584475030698053
MSE_test: 0.9581377498537527


In [247]:
print('RMSE_train:', root_mean_squared_error(y_train, y_train_pred))
print("RMSE_test:", root_mean_squared_error(y_test, y_test_pred))

RMSE_train: 0.7472934517776837
RMSE_test: 0.9788451102466379


In [248]:
print('R2_train:', r2_score(y_train, y_train_pred))
print("R2_test:", r2_score(y_test, y_test_pred))

R2_train: 0.6199831644607661
R2_test: 0.1809266856045133


<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Training with Random Forest Regressor</div>

In [249]:
rf_model = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [250]:
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

In [251]:
print('MAE_train_rf:', mean_absolute_error(y_train, y_train_pred_rf))
print("MAE_test_rf:", mean_absolute_error(y_test, y_test_pred_rf))

MAE_train_rf: 0.20567924037460975
MAE_test_rf: 0.5305848074921957


In [252]:
print('MSE_train_rf:', mean_squared_error(y_train, y_train_pred_rf))
print("MSE_test_rf:", mean_squared_error(y_test, y_test_pred_rf))

MSE_train_rf: 0.08118569328824143
MSE_test_rf: 0.5703193361082204


In [253]:
print('RMSE_train_rf:', root_mean_squared_error(y_train, y_train_pred_rf))
print("RMSE_test_rf:", root_mean_squared_error(y_test, y_test_pred_rf))

RMSE_train_rf: 0.28493103251180174
RMSE_test_rf: 0.7551948994188324


In [254]:
print('R2_train_rf:', r2_score(y_train, y_train_pred_rf))
print("R2_test_rf:", r2_score(y_test, y_test_pred_rf))

R2_train_rf: 0.9447541083363034
R2_test_rf: 0.5124570042655187


<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Training with XGBRegressor</div>

In [255]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [256]:
y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

In [257]:
print('MAE_train_xgb:', mean_absolute_error(y_train, y_train_pred_xgb))
print("MAE_test_xgb:", mean_absolute_error(y_test, y_test_pred_xgb))

MAE_train_xgb: 0.29861544186731387
MAE_test_xgb: 0.565421012248243


In [258]:
print('MSE_train_xgb:', mean_squared_error(y_train, y_train_pred_xgb))
print("MSE_test_xgb:", mean_squared_error(y_test, y_test_pred_xgb))

MSE_train_xgb: 0.14577902525445824
MSE_test_xgb: 0.6146296552874091


In [259]:
print('RMSE_train_xgb:', root_mean_squared_error(y_train, y_train_pred_xgb))
print("RMSE_test_xgb:", root_mean_squared_error(y_test, y_test_pred_xgb))

RMSE_train_xgb: 0.38181019532544996
RMSE_test_xgb: 0.7839831983450979


In [260]:
print('R2_train_xgb:', r2_score(y_train, y_train_pred_xgb))
print("R2_test_xgb:", r2_score(y_test, y_test_pred_xgb))

R2_train_xgb: 0.9007991197728239
R2_test_xgb: 0.4745778997238599


In [261]:
df_final.columns

Index(['title', 'vote_average', 'overview', 'budget', 'revenue', 'runtime',
       'popularity', 'vote_count', 'release_year', 'release_month',
       'num_genres', 'num_keywords', 'num_cast', 'genres_list',
       'keywords_list', 'overview_length', 'overview_cleaned'],
      dtype='object')

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b> Training and prediction using Nueral Networks models
</b></div>

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Training with GRU</div>

In [262]:
# MAX_WORDS = 5000  # Number of most common words 
# MAX_LEN = 150     # length of the sequence

# tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
# tokenizer.fit_on_texts(df_final['overview_cleaned'])
# sequences = tokenizer.texts_to_sequences(df_final['overview_cleaned'])
# X_text = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [263]:
# X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(
#     X_text, X_num_scaled, y, test_size=0.2, random_state=42
# )

In [264]:
# def build_enhanced_gru_model(vocab_size, max_len, num_features, embedding_dim=128):
#     """
#     - Bidirectional GRU
#     - Multiple GRU layers
#     - Batch Normalization
#     - Dropout
#     - Dense layers
#     """
    
#     # Text Input Branch
#     text_input = Input(shape=(max_len,), name='text_input')
    
#     # Embedding Layer
#     embedding = Embedding(
#         input_dim=vocab_size,
#         output_dim=embedding_dim,
#         input_length=max_len,
#         mask_zero=True,
#         name='embedding'
#     )(text_input)
    
#     # Bidirectional GRU Layers
#     gru1 = Bidirectional(
#         GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
#         name='bigru_1'
#     )(embedding)
#     gru1 = BatchNormalization()(gru1)
    
#     gru2 = Bidirectional(
#         GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
#         name='bigru_2'
#     )(gru1)
#     gru2 = BatchNormalization()(gru2)
    
#     # Global Pooling (Average + Max)
#     avg_pool = GlobalAveragePooling1D()(gru2)
#     max_pool = GlobalMaxPooling1D()(gru2)
#     text_features = Concatenate()([avg_pool, max_pool])
    
#     # Numerical Input Branch
#     num_input = Input(shape=(num_features,), name='num_input')
#     num_dense = Dense(64, activation='relu')(num_input)
#     num_dense = BatchNormalization()(num_dense)
#     num_dense = Dropout(0.3)(num_dense)
    
#     # Merge Both Branches
#     merged = Concatenate()([text_features, num_dense])
    
#     # Dense Layers
#     dense1 = Dense(256, activation='relu')(merged)
#     dense1 = BatchNormalization()(dense1)
#     dense1 = Dropout(0.4)(dense1)
    
#     dense2 = Dense(128, activation='relu')(dense1)
#     dense2 = BatchNormalization()(dense2)
#     dense2 = Dropout(0.3)(dense2)
    
#     dense3 = Dense(64, activation='relu')(dense2)
#     dense3 = BatchNormalization()(dense3)
#     dense3 = Dropout(0.2)(dense3)
    
#     # Output Layer
#     output = Dense(1, activation='linear', name='output')(dense3)
    
#     # Create Model
#     model = Model(inputs=[text_input, num_input], outputs=output)
    
#     return model

In [265]:
# vocab_size = min(MAX_WORDS, len(tokenizer.word_index) + 1)
# num_features = X_train_num.shape[1]

# model = build_enhanced_gru_model(vocab_size, MAX_LEN, num_features)

In [266]:
# optimizer = Adam(learning_rate=0.001)
# model.compile(
#     optimizer=optimizer,
#     loss='huber',
#     metrics=['mae', 'mse']
# )
# model.summary()

In [267]:
# callbacks = [
#     EarlyStopping(
#         monitor='val_loss',
#         patience=15,
#         restore_best_weights=True,
#         verbose=1
#     ),
#     ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=0.5,
#         patience=7,
#         min_lr=1e-7,
#         verbose=1
#     ),
#     ModelCheckpoint(
#         'best_gru_model.keras',
#         monitor='val_loss',
#         save_best_only=True,
#         verbose=1
#     )
# ]

In [268]:
# history = model.fit(
#     [X_train_text, X_train_num],
#     y_train,
#     validation_split=0.15,
#     epochs=100,
#     batch_size=32,
#     callbacks=callbacks,
#     verbose=1
# )

In [269]:
# y_train_pred = model.predict([X_train_text, X_train_num]).flatten()
# y_test_pred = model.predict([X_test_text, X_test_num]).flatten()

In [270]:
# print("Training Set Performance:")
# print(f"MAE:  {mean_absolute_error(y_train, y_train_pred):.4f}")
# print(f"MSE:  {mean_squared_error(y_train, y_train_pred):.4f}")
# print(f"RMSE: {root_mean_squared_error(y_train, y_train_pred):.4f}")
# print(f"R²:   {r2_score(y_train, y_train_pred):.4f}")

# print("\nTesting Set Performance:")
# print(f"MAE:  {mean_absolute_error(y_test, y_test_pred):.4f}")
# print(f"MSE:  {mean_squared_error(y_test, y_test_pred):.4f}")
# print(f"RMSE: {root_mean_squared_error(y_test, y_test_pred):.4f}")
# print(f"R²:   {r2_score(y_test, y_test_pred):.4f}")

In [271]:
# fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# # Loss
# axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
# axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
# axes[0].set_title('Model Loss', fontsize=14, fontweight='bold')
# axes[0].set_xlabel('Epoch')
# axes[0].set_ylabel('Loss')
# axes[0].legend()
# axes[0].grid(True, alpha=0.3)

# # MAE
# axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
# axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
# axes[1].set_title('Model MAE', fontsize=14, fontweight='bold')
# axes[1].set_xlabel('Epoch')
# axes[1].set_ylabel('MAE')
# axes[1].legend()
# axes[1].grid(True, alpha=0.3)

# plt.tight_layout()
# plt.show()

In [272]:
# sample_indices = np.random.choice(len(y_test), 10, replace=False)
# print("Sample Predictions vs Actual:")
# print("-" * 60)
# print(f"{'Actual':<15} {'Predicted':<15} {'Difference':<15}")
# print("-" * 60)

# for idx in sample_indices:
#     actual = y_test.iloc[idx]
#     predicted = y_test_pred[idx]
#     diff = abs(actual - predicted)
#     print(f"{actual:<15.2f} {predicted:<15.2f} {diff:<15.2f}")

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:7px; color:white; margin:0; font-size:110%; font-family:Pacifico; background-color:#0f7a7a; overflow:hidden"><b>Hyperparameter Tuning
</b></div>

<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Tuning Random Forest Regressor</div>

In [273]:
param_grid = {
    'n_estimators': [200, 400],
    'max_depth': [None, 20, 40],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [275]:
grid = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1
)

In [276]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
             param_grid={'max_depth': [None, 20, 40],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [200, 400]},
             scoring='neg_mean_absolute_error', verbose=1)

In [277]:
best_rf = grid.best_estimator_
print(grid.best_params_)

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}


In [278]:
y_test_pred_rf_tun = best_rf.predict(X_test)
y_train_pred_rf_tun = best_rf.predict(X_train)

In [279]:
# Train metrics
print("Train MAE:", mean_absolute_error(y_train, y_train_pred_rf_tun))
print("Train MSE:", mean_squared_error(y_train, y_train_pred_rf_tun))
print("Train RMSE:", root_mean_squared_error(y_train, y_train_pred_rf_tun))
print("Train R2:", r2_score(y_train, y_train_pred_rf_tun))

Train MAE: 0.20254656607700333
Train MSE: 0.07878262041493231
Train RMSE: 0.2806824191411573
Train R2: 0.9463893706373526


In [280]:
# Test metrics
print("Test MAE:", mean_absolute_error(y_test, y_test_pred_rf_tun))
print("Test MSE:", mean_squared_error(y_test, y_test_pred_rf_tun))
print("Test RMSE:", root_mean_squared_error(y_test, y_test_pred_rf_tun))
print("Test R2:", r2_score(y_test, y_test_pred_rf_tun))

Test MAE: 0.5258998439125914
Test MSE: 0.5622503895031221
Test RMSE: 0.749833574537125
Test R2: 0.5193548212448902


<div style="padding:10px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:10px;background-color:#215f95;;overflow:hidden;font-weight:501;font-family:magra">Save model</div>

In [284]:
joblib.dump(best_rf, 'best_rf_model.pkl')

['best_rf_model.pkl']

In [285]:
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [288]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']